In [1]:
!python -V

Python 3.9.13


In [2]:
#!wget https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2022-01.parquet
#!wget https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2022-02.parquet

In [3]:
#!pip install pyarrow fastparquet

In [4]:
import pandas as pd
import pickle

import seaborn as sns
import matplotlib.pyplot as plt

from scipy import stats

from sklearn.preprocessing import OneHotEncoder

%matplotlib inline

import warnings
warnings.filterwarnings("ignore")

In [5]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
#from sklearn.linear_model import Lasso
#from sklearn.linear_model import Ridge

from sklearn.metrics import mean_squared_error

In [6]:
df = pd.read_parquet('./yellow_tripdata_2022-01.parquet')
df.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,1,2022-01-01 00:35:40,2022-01-01 00:53:29,2.0,3.80,1.0,N,142,236,1,14.5,3.0,0.5,3.65,0.0,0.3,21.95,2.5,0.0
1,1,2022-01-01 00:33:43,2022-01-01 00:42:07,1.0,2.10,1.0,N,236,42,1,8.0,0.5,0.5,4.00,0.0,0.3,13.30,0.0,0.0
2,2,2022-01-01 00:53:21,2022-01-01 01:02:19,1.0,0.97,1.0,N,166,166,1,7.5,0.5,0.5,1.76,0.0,0.3,10.56,0.0,0.0
3,2,2022-01-01 00:25:21,2022-01-01 00:35:23,1.0,1.09,1.0,N,114,68,2,8.0,0.5,0.5,0.00,0.0,0.3,11.80,2.5,0.0
4,2,2022-01-01 00:36:48,2022-01-01 01:14:20,1.0,4.30,1.0,N,68,163,1,23.5,0.5,0.5,3.00,0.0,0.3,30.30,2.5,0.0


### Question 1: Number of columns in Jan 2022 Yellow Taxi Trip data

In [7]:
len(df.columns)

19

### Question 2:  Standard deviation of the trips duration in Jan 2022 Yellow Taxi Trip data

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2463931 entries, 0 to 2463930
Data columns (total 19 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   VendorID               int64         
 1   tpep_pickup_datetime   datetime64[ns]
 2   tpep_dropoff_datetime  datetime64[ns]
 3   passenger_count        float64       
 4   trip_distance          float64       
 5   RatecodeID             float64       
 6   store_and_fwd_flag     object        
 7   PULocationID           int64         
 8   DOLocationID           int64         
 9   payment_type           int64         
 10  fare_amount            float64       
 11  extra                  float64       
 12  mta_tax                float64       
 13  tip_amount             float64       
 14  tolls_amount           float64       
 15  improvement_surcharge  float64       
 16  total_amount           float64       
 17  congestion_surcharge   float64       
 18  airport_fee           

In [9]:
df.tpep_dropoff_datetime = pd.to_datetime(df.tpep_dropoff_datetime)
df.tpep_pickup_datetime = pd.to_datetime(df.tpep_pickup_datetime)

In [10]:
df['duration'] = df['tpep_dropoff_datetime'] - df['tpep_pickup_datetime']
df.duration = df.duration.apply(lambda duration: duration.total_seconds()/60)

In [11]:
round(df['duration'].std(),2)

46.45

### Question 3: Fraction of the records left after dropping the outliers

In [12]:
df["duration"].describe(percentiles=[0.9, 0.93, 0.95, 0.98, 0.99]).apply(lambda x: format(x, 'f'))

count    2463931.000000
mean          14.212203
std           46.445305
min        -3442.400000
50%           10.183333
90%           24.633333
93%           28.300000
95%           31.933333
98%           42.150000
99%           50.850000
max         8513.183333
Name: duration, dtype: object

In [13]:
df["duration"].std()

46.44530513776499

In [14]:
stats.iqr(df["duration"])

9.850000000000001

In [15]:
df["duration"].mean()

14.212202918831741

In [16]:
upper = 1.5 * stats.iqr(df["duration"]) + df["duration"].quantile(0.75)
upper

30.94166666666667

In [17]:
df["duration"].isna().sum()

0

In [18]:
round((1 - (df[df["duration"] >= upper].count().iloc[0] / df["duration"].count())),2)

0.95

### Question 4: Dimensionality after OHE

In [19]:
df.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee,duration
0,1,2022-01-01 00:35:40,2022-01-01 00:53:29,2.0,3.80,1.0,N,142,236,1,14.5,3.0,0.5,3.65,0.0,0.3,21.95,2.5,0.0,17.816667
1,1,2022-01-01 00:33:43,2022-01-01 00:42:07,1.0,2.10,1.0,N,236,42,1,8.0,0.5,0.5,4.00,0.0,0.3,13.30,0.0,0.0,8.400000
2,2,2022-01-01 00:53:21,2022-01-01 01:02:19,1.0,0.97,1.0,N,166,166,1,7.5,0.5,0.5,1.76,0.0,0.3,10.56,0.0,0.0,8.966667
3,2,2022-01-01 00:25:21,2022-01-01 00:35:23,1.0,1.09,1.0,N,114,68,2,8.0,0.5,0.5,0.00,0.0,0.3,11.80,2.5,0.0,10.033333
4,2,2022-01-01 00:36:48,2022-01-01 01:14:20,1.0,4.30,1.0,N,68,163,1,23.5,0.5,0.5,3.00,0.0,0.3,30.30,2.5,0.0,37.533333


In [20]:
df.iloc[:, :-1].shape[1]

19

In [21]:
pd.get_dummies(df).shape[1]

21

In [22]:
pd.get_dummies(df).shape[1] - df.iloc[:, :-1].shape[1]

2

### Question 5: RMSE on train

### Question 6: RMSE on validation

### myTEST

In [23]:
((df["duration"] >= 1) & (df["duration"] <= 60)).mean()

0.9827547930522406

In [24]:
df.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee,duration
0,1,2022-01-01 00:35:40,2022-01-01 00:53:29,2.0,3.80,1.0,N,142,236,1,14.5,3.0,0.5,3.65,0.0,0.3,21.95,2.5,0.0,17.816667
1,1,2022-01-01 00:33:43,2022-01-01 00:42:07,1.0,2.10,1.0,N,236,42,1,8.0,0.5,0.5,4.00,0.0,0.3,13.30,0.0,0.0,8.400000
2,2,2022-01-01 00:53:21,2022-01-01 01:02:19,1.0,0.97,1.0,N,166,166,1,7.5,0.5,0.5,1.76,0.0,0.3,10.56,0.0,0.0,8.966667
3,2,2022-01-01 00:25:21,2022-01-01 00:35:23,1.0,1.09,1.0,N,114,68,2,8.0,0.5,0.5,0.00,0.0,0.3,11.80,2.5,0.0,10.033333
4,2,2022-01-01 00:36:48,2022-01-01 01:14:20,1.0,4.30,1.0,N,68,163,1,23.5,0.5,0.5,3.00,0.0,0.3,30.30,2.5,0.0,37.533333


In [25]:
#df = df.dropna(inplace=True)

In [26]:
df.isna().sum()

VendorID                     0
tpep_pickup_datetime         0
tpep_dropoff_datetime        0
passenger_count          71503
trip_distance                0
RatecodeID               71503
store_and_fwd_flag       71503
PULocationID                 0
DOLocationID                 0
payment_type                 0
fare_amount                  0
extra                        0
mta_tax                      0
tip_amount                   0
tolls_amount                 0
improvement_surcharge        0
total_amount                 0
congestion_surcharge     71503
airport_fee              71503
duration                     0
dtype: int64

In [27]:
categorical = ["PULocationID", "DOLocationID"]
numerical = ["trip_distance", "total_amount"]

In [28]:
df[categorical] = df[categorical].astype(str)

In [29]:
df[categorical].dtypes

PULocationID    object
DOLocationID    object
dtype: object

In [30]:
df[numerical].dtypes

trip_distance    float64
total_amount     float64
dtype: object

In [31]:
dv = DictVectorizer()

In [32]:
train_data = df[categorical + numerical].dropna().to_dict(orient='records')

In [33]:
X_train = dv.fit_transform(train_data)

In [34]:
X_train

<2463931x520 sparse matrix of type '<class 'numpy.float64'>'
	with 9855724 stored elements in Compressed Sparse Row format>

In [35]:
y_train = df["duration"].values

In [36]:
y_train

array([17.81666667,  8.4       ,  8.96666667, ..., 11.        ,
       12.05      , 27.        ])

In [37]:
lr = LinearRegression()

In [38]:
lr.fit(X_train, y_train)

LinearRegression()

In [39]:
y_pred = lr.predict(X_train)

In [40]:
y_pred

array([10.88980323, 17.67442775, 15.81251291, ..., 11.6188376 ,
       12.42377237, 33.8320479 ])

In [41]:
model_df = pd.DataFrame(data={"train": y_train, "pred": y_pred})

In [42]:
model_df.head()

,train,pred
0,17.816667,10.889803
1,8.400000,17.674428
2,8.966667,15.812513
3,10.033333,11.429759
4,37.533333,12.944294


In [43]:
rmse = mean_squared_error(y_train, y_pred, squared=False)
rmse

46.03547733387083

In [44]:
with open("models/linear_regression.bin", "wb") as fout:
    pickle.dump((dv, lr), fout)